In [1]:
import os
os.chdir('../')

In [2]:
import sys
sys.path.append('./')

In [3]:
from box import Box
from pytorch_lightning import Trainer
import numpy as np

from src.SLR.data import VideoDataModule
from src.SLR.models import SLR_Lightning

/home/wpzhang/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
cfg = Box.from_yaml(open('configs/SLR/cfg.yaml', "r").read())

In [5]:
dataset_cfg = cfg.dataset_args
model_cfg = cfg.model_args
optimizer_cfg = cfg.optimizer_args
trainer_cfg = cfg.trainer_args

In [6]:
gloss_dict = np.load(os.path.join(dataset_cfg.info_dir, "gloss_dict.npy"), allow_pickle=True).item()

In [7]:
slr_model = SLR_Lightning(gloss_dict=gloss_dict, **model_cfg, **optimizer_cfg)
dm = VideoDataModule(gloss_dict=gloss_dict, **dataset_cfg)

/home/wpzhang/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/wpzhang/miniconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
trainer = Trainer(**trainer_cfg)
trainer.fit(slr_model, dm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/wpzhang/Documents/SignLanguageTranslation/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | SLRModel | 33.0 M
-----------------------------------
33.0 M    Trainable params
0         Non-trainable params
33.0 M    Total params
131.894   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]Valset
Sanity Checking DataLoader 0:  50%|█████     | 1/2 [00:01<00:01,  1.25s/it]

/home/wpzhang/miniconda3/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 2. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Trainset                                                                   
Epoch 0:   0%|          | 0/3105 [00:00<?, ?it/s] 

OutOfMemoryError: CUDA out of memory. Tried to allocate 108.00 MiB (GPU 0; 5.80 GiB total capacity; 5.26 GiB already allocated; 27.94 MiB free; 5.59 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF